# Locating the available data
In this notebook, we will use KD-Tree to locate all the biochemical data in the eReef dataset that is within 50km radius from each position in Debbie's path. Once we have this information, we will use it to calculate the descriptive statistics of each of the variables within the path (temp, Secchi and chlorophyll-a). 

We will do these calculations for 3 periods: before, after, and during cyclone Debbie, so we can compate the impacts of the cyclone to normal condisions. 

We first create a combined file of all statistics for the entire period (01/03/2017-22/04/2017), then we split them to 

(1) during (9 days, 22/03/2017-31/03/2017), 

(2) before (01/03/2017-22/03/2017) and then 
(3) after (01/04/2017-22/04/2017). 

In [2]:
# load the necessary packages 

import os
import scipy
import shutil
import numpy as np
import pandas as pd
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))

import plotly.express as px
import geopandas as gpd
from scipy.spatial import cKDTree

import imageio
import os
import io
import requests

import datetime as dt
from dateutil.relativedelta import *

import netCDF4
from netCDF4 import Dataset, num2date

import cmocean

import seaborn as sns
import pymannkendall as mk

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from matplotlib import pyplot as plt
%matplotlib inline

from PIL import Image  # To create gifs
from shapely.geometry import Point
from shapely.geometry import LineString

In [3]:
# loading the csv with Debbie's path 
df = pd.read_csv('debbie.csv',parse_dates=['datetime'])
time = df['datetime'].dt.tz_localize(None)
df

,seq,datetime,lon,wind_kts,wind_mps,year,pressure,lat,name
0,13,2017-03-22 00:00:00+00:00,152.00,14.967616,7.7,2016,1006,-13.20,DEBBIE
1,13,2017-03-22 06:00:00+00:00,151.70,14.967616,7.7,2016,1010,-12.00,DEBBIE
2,13,2017-03-22 12:00:00+00:00,154.00,14.967616,7.7,2016,1011,-12.10,DEBBIE
3,13,2017-03-23 00:00:00+00:00,152.60,20.021616,10.3,2016,1008,-13.30,DEBBIE
4,13,2017-03-23 06:00:00+00:00,151.60,20.021616,10.3,2016,1000,-14.10,DEBBIE
5,13,2017-03-23 12:00:00+00:00,151.50,20.021616,10.3,2016,1000,-14.50,DEBBIE
6,13,2017-03-23 18:00:00+00:00,151.30,25.075616,12.9,2016,1000,-15.30,DEBBIE
7,13,2017-03-24 00:00:00+00:00,151.30,25.075616,12.9,2016,999,-15.90,DEBBIE
8,13,2017-03-24 06:00:00+00:00,151.60,25.075616,12.9,2016,996,-16.40,DEBBIE
9,13,2017-03-24 12:00:00+00:00,151.80,29.935231,15.4,2016,996,-16.80,DEBBIE


In [4]:
# Creating a new dataset with only lat, lon and time for Debbie
path = df[['lon', 'lat', 'datetime']]  # Select the columns
path

,lon,lat,datetime
0,152.00,-13.20,2017-03-22 00:00:00+00:00
1,151.70,-12.00,2017-03-22 06:00:00+00:00
2,154.00,-12.10,2017-03-22 12:00:00+00:00
3,152.60,-13.30,2017-03-23 00:00:00+00:00
4,151.60,-14.10,2017-03-23 06:00:00+00:00
5,151.50,-14.50,2017-03-23 12:00:00+00:00
6,151.30,-15.30,2017-03-23 18:00:00+00:00
7,151.30,-15.90,2017-03-24 00:00:00+00:00
8,151.60,-16.40,2017-03-24 06:00:00+00:00
9,151.80,-16.80,2017-03-24 12:00:00+00:00


In [8]:
# open the biogeochemical data from eReefs (netCDF file) 
month = 3
year = 2017
netCDF_datestr = str(year)+'-'+format(month, '02')

# GBR4 BIO
biofiles = ["https://thredds.ereefs.aims.gov.au/thredds/dodsC/GBR4_H2p0_B3p1_Cq3b_Dhnd/daily.nc"]

ds_bio = xr.open_mfdataset(biofiles)

# Define the time range (for the duration of the cyclone only)
start_date = '2017-03-01'
end_date = '2017-04-22'

# Slice the dataset based on the time range
ds_bio_slice = ds_bio.sel(time=slice(start_date, end_date), k=-1)
ds_bio_slice

<xarray.Dataset>
Dimensions:          (latitude: 723, longitude: 491, time: 53)
Coordinates:
    zc               float64 dask.array<chunksize=(), meta=np.ndarray>
  * latitude         (latitude) float64 -28.7 -28.67 -28.64 ... -7.066 -7.036
  * longitude        (longitude) float64 142.2 142.2 142.2 ... 156.8 156.8 156.9
  * time             (time) datetime64[ns] 2017-03-01T02:00:00 ... 2017-04-22...
Data variables: (12/108)
    TOTAL_NITROGEN   (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    alk              (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    BOD              (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    Chl_a_sum        (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    CO32             (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    DetBL_N          (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    ...               ...
    SGH_N            (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    SGH_N_pr         (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    SGHROOT_N        (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    SGROOT_N         (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    TSSM             (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
    Zenith2D         (time, latitude, longitude) float32 dask.array<chunksize=(53, 723, 491), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                   CF-1.0
    NCO:                           netCDF Operators version 4.7.7 (Homepage =...
    RunID:                         2
    _CoordSysBuilder:              ucar.nc2.dataset.conv.CF1Convention
    aims_ncaggregate_buildDate:    2021-09-07T22:58:30+10:00
    aims_ncaggregate_datasetId:    products__ncaggregate__ereefs__GBR4_H2p0_B...
    ...                            ...
    metadata_link:                 https://eatlas.org.au/data/uuid/61f3a6df-2...
    paramfile:                     /home/bai155/EMS_solar2/gbr4_H2p0_B3p1_Cb/...
    paramhead:                     eReefs 4 km grid. SOURCE Catchments with 2...
    technical_guide_link:          https://eatlas.org.au/pydio/public/aims-er...
    technical_guide_publish_date:  2020-08-18
    title:                         eReefs AIMS-CSIRO GBR4 BioGeoChemical 3.1 ...

In [9]:
# creating a dataset with only temperature, chlorophyll-a and secchi for the 10 days of the cyclone 

# Select the variables of interest
selected_vars = ds_bio_slice[['Chl_a_sum', 'temp', 'Secchi']]

# Create a new dataset with these variables
new_bio = xr.Dataset(
    {
        'Chl_a_sum': selected_vars['Chl_a_sum'],
        'temp': selected_vars['temp'],
        'Secchi': selected_vars['Secchi']
    }
)

# Creating the KD-Tree
We are going to use KD-tree to locate the datapoints available near the cyclone path. To do that, we need to create a mesh comprised of the new_bio dataset. Each point of data in the new_bio dataset has its own coordinates, in order to find the data within 50 km of cyclone Debbie's path, we need to organise the data in a grid. To do that, we create a mesh based on the latitude & longitude of the data in new_bio.

In [120]:
# Extracting the lon lat of the data
lon=new_bio.longitude.values
lat=new_bio.latitude.values

# Build the mesh based on lon lat
mlon, mlat = np.meshgrid(lon,lat)
mlonlat = np.dstack([mlon.flatten(), mlat.flatten()])[0]

# The shape of 'mlon' and 'mlat' is the same as the shape of the new_bio dataset 
# we create a Numpy list 'mlonlat' that is flattened for the cKDTree

mlon.shape        # (723,491)
mlonlat.shape     # (354993, 2)

(354993, 2)

Now, we create the KD-tree based on the new_bio mech, and we add the cyclone path 

In [124]:
# Building the cKDtree
tree = scipy.spatial.cKDTree(mlonlat)

# Cyclone path (converting to Numpy list)
cyclone_pos = np.zeros((len(path),2))
cyclone_pos[:,0] = path['lon']
cyclone_pos[:,1] = path['lat']

To find all avaialble data for all cyclone positions, we create a loop that will find data (in new_bio) that is within 50 km the cyclone path. 

The path dataset contains 39 positions of the cyclone throughout its activity, each with longitude and latitude. 
The loop will locate the  IDs (the index number) of the data in new_bio, within 50km of each of the 39 positions of the cyclone.

In [128]:
# Creating the loop:
for c in range(len(cyclone_pos)):

    ids = tree.query_ball_point(cyclone_pos[c,:], r=0.45)
    idlon = []
    idlat = []
    for k in range(len(ids)):
        ptlon = mlonlat[ids[k],0]
        ptlat = mlonlat[ids[k],1]
        idlon.append(np.where(ptlon == lon)[0][0])
        idlat.append(np.where(ptlat == lat)[0][0])

# Saving the IDs for each position as a separate file (total 39 files, each file containing all the IDs of new_bio)
    data = {'idlon': idlon,
            'idlat': idlat}
    df = pd.DataFrame(data)
    df.to_csv('idclosetocycBDA/pos'+str(c)+'.csv',index=False)

Now that we have the IDs of all the data within 50km of the cyclone path, we need to compute the mean, max and min for each of the selected variables (temp, secchi and chlorophyll-a). We will do that for before, during and after the cyclone. 

In [132]:
# List of variables to process
variables = ['temp', 'Secchi', 'Chl_a_sum']

# Create a dictionary to store statistics for each variable and position
all_stats = {f'{var}_{pos}': {'Mean': [], 'Max': [], 'Min': []} for var in variables for pos in range(37) if not (25 <= pos <= 34)}

# Loop through each position file, excluding: 
# (1) pos0 - pos3 and pos37 to pos39 - because there is no data in these poisitons
# (2) pos25 to pos34 - because the cyclone is above land in these poisitons 

for pos in range(37):
    if pos in range(4) or 25 <= pos <= 34:
        continue
    
# Read the corresponding CSV file
    idpos = pd.read_csv(f'idclosetocycBDA/pos{pos}.csv')
    
# Loop through each variable
    for var in variables:
# Creating lists to store mean, max, and min values for all days in the range 
        daily_means = []
        daily_maxs = []
        daily_mins = []
        
# Loop through each day (time index from 0 to 52)
        for day in range(52):
# Extract values for the specific day and position
            values = getattr(new_bio, var).isel(time=day).values[idpos['idlat'].values, idpos['idlon'].values]
            
# Calculate statistics, ignoring NaNs
            mean_val = np.nanmean(values)
            max_val = np.nanmax(values)
            min_val = np.nanmin(values)
            
# Append results to the lists
            daily_means.append(mean_val)
            daily_maxs.append(max_val)
            daily_mins.append(min_val)
        
# Store the statistics for the current variable and position
        all_stats[f'{var}_{pos}']['Mean'] = daily_means
        all_stats[f'{var}_{pos}']['Max'] = daily_maxs
        all_stats[f'{var}_{pos}']['Min'] = daily_mins

# Create a DataFrame combining mean, max, and min values for all variables
data = {}
for var in variables:
    for pos in range(37):
        if pos in range(4) or 25 <= pos <= 34:
            continue
        data[f'{var}_pos{pos}_Mean'] = all_stats[f'{var}_{pos}']['Mean']
        data[f'{var}_pos{pos}_Max'] = all_stats[f'{var}_{pos}']['Max']
        data[f'{var}_pos{pos}_Min'] = all_stats[f'{var}_{pos}']['Min']

# Create DataFrame
df_all_stats = pd.DataFrame(data, index=[f'Day {i+1}' for i in range(52)])

# Transpose DataFrame to have days as columns and variables as rows
df_all_stats_T = df_all_stats.T

# Save the transposed DataFrame to a new CSV file
df_all_stats_T.to_csv('combined_stats_BDA.csv')

print('Combined statistics for temp, Secchi, and Chl_a_sum saved to combined_stats_BDA.csv')

KeyboardInterrupt: 

In [138]:
# Splitting the DataFrame based on the 3 periods:
combined_stats_BDA = pd.read_csv('combined_stats_BDA.csv')

# We will organise the Data Frame so that it is more convinent to work with and perform calculations and plotting
# Starting with splitting the first column to 3 separate columns, based on the variable, position, and statistics

parse = combined_stats_BDA['Unnamed: 0'].iloc[0]
parse_split = parse.split("_")

var_name = []
position = []
stats = []
for var in range(len(combined_stats_BDA)):
    parse = combined_stats_BDA['Unnamed: 0'].iloc[var]
    parse_split = parse.split("_")
    parse_split
    var_name.append(parse_split[0])
    position.append(parse_split[-2])
    stats.append(parse_split[-1])

combined_stats_BDA['var_name']  = var_name
combined_stats_BDA['position']  = position
combined_stats_BDA['stats']  = stats
combined_stats_BDA

,Unnamed: 0,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,...,Day 46,Day 47,Day 48,Day 49,Day 50,Day 51,Day 52,var_name,position,stats
0,temp_pos4_Mean,29.545475,29.832327,29.904980,30.037544,30.336048,30.582610,30.167805,30.107754,30.702030,...,28.934351,28.954609,28.743603,28.695757,28.702540,28.719805,28.707952,temp,pos4,Mean
1,temp_pos4_Max,29.721504,29.982054,30.097982,30.352020,30.494875,30.755377,30.405440,30.300330,30.897224,...,29.115461,29.143010,28.895136,28.868390,28.875546,28.990524,29.040077,temp,pos4,Max
2,temp_pos4_Min,29.286646,29.582176,29.706150,29.798850,30.130170,30.399452,29.990221,29.909258,30.531471,...,28.740602,28.710018,28.615911,28.544220,28.445950,28.523098,28.470590,temp,pos4,Min
3,temp_pos5_Mean,29.517980,29.762335,29.859703,29.948300,30.254803,30.552685,30.186838,30.209017,30.735277,...,28.816100,28.827497,28.730782,28.631142,28.548365,28.556920,28.529724,temp,pos5,Mean
4,temp_pos5_Max,29.721504,29.982054,30.117926,30.307375,30.540840,30.820726,30.526466,30.506773,30.978699,...,28.998184,29.094818,28.851795,28.824785,28.829636,28.793621,28.790546,temp,pos5,Max
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Chl_a_sum_pos35_Max,0.029038,0.027634,0.025081,0.028265,0.032324,0.031930,0.046667,0.048954,0.046206,...,0.096341,0.106278,0.108281,0.092855,0.091042,0.085142,0.070077,Chl,pos35,Max
203,Chl_a_sum_pos35_Min,0.020183,0.021320,0.019288,0.017746,0.018172,0.017650,0.017683,0.018421,0.018980,...,0.038202,0.038159,0.039021,0.039991,0.041459,0.043528,0.041739,Chl,pos35,Min
204,Chl_a_sum_pos36_Mean,0.029552,0.029112,0.028681,0.031548,0.031251,0.024817,0.028993,0.035033,0.037237,...,0.075054,0.062919,0.065821,0.064889,0.068187,0.069966,0.078630,Chl,pos36,Mean
205,Chl_a_sum_pos36_Max,0.036832,0.034815,0.034480,0.041938,0.046589,0.041988,0.036978,0.051341,0.051758,...,0.195269,0.128603,0.152264,0.167157,0.182100,0.187512,0.190625,Chl,pos36,Max


In [140]:
# Splitting the DataFrame based on the 3 periods:
# before:
before_columns = combined_stats_BDA.iloc[:, 1:22]
keep_columns1 = combined_stats_BDA[['var_name', 'position', 'stats']]
before = pd.concat([keep_columns1, before_columns], axis=1)
before.to_csv('combined_stats_BEFORE.csv')

In [142]:
# during 
during_columns = combined_stats_BDA.iloc[:, 22:31]
keep_columns2 = combined_stats_BDA[['var_name', 'position', 'stats']]
during = pd.concat([keep_columns2, during_columns], axis=1)
during.to_csv('combined_stats_DURING.csv')

In [144]:
# after:
after_columns = combined_stats_BDA.iloc[:, 31:53]
keep_columns3 = combined_stats_BDA[['var_name', 'position', 'stats']]
after = pd.concat([keep_columns3, after_columns], axis=1)
after.to_csv('combined_stats_AFTER.csv')